## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavaratti,IN,10.5669,72.6420,82.85,75,100,10.80,overcast clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,46.45,79,96,0.96,overcast clouds
2,2,Flinders,AU,-34.5833,150.8552,53.06,85,63,1.01,broken clouds
3,3,Kodiak,US,57.7900,-152.4072,53.22,59,19,6.55,few clouds
4,4,Castro,BR,-24.7911,-50.0119,68.86,61,100,4.38,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavaratti,IN,10.5669,72.6420,82.85,75,100,10.80,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,80.58,71,90,1.99,overcast clouds
9,9,Atuona,PF,-9.8000,-139.0333,79.70,64,71,8.37,broken clouds
16,16,Mahibadhoo,MV,3.7833,72.9667,83.03,76,93,14.92,overcast clouds
22,22,Rikitea,PF,-23.1203,-134.9692,76.71,84,100,15.05,overcast clouds
24,24,Novo Aripuana,BR,-5.1206,-60.3797,84.69,69,100,1.54,overcast clouds
28,28,Praia,CV,14.9215,-23.5087,77.54,69,20,21.85,few clouds
30,30,Kaele,CM,10.1092,14.4508,87.87,44,100,5.57,overcast clouds
38,38,Ujjain,IN,23.1833,75.7667,86.79,49,96,8.14,overcast clouds
39,39,Victoria,HK,22.2855,114.1577,82.33,85,44,4.00,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavaratti,IN,82.85,overcast clouds,10.5669,72.6420,
7,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,
9,Atuona,PF,79.70,broken clouds,-9.8000,-139.0333,
16,Mahibadhoo,MV,83.03,overcast clouds,3.7833,72.9667,
22,Rikitea,PF,76.71,overcast clouds,-23.1203,-134.9692,
24,Novo Aripuana,BR,84.69,overcast clouds,-5.1206,-60.3797,
28,Praia,CV,77.54,few clouds,14.9215,-23.5087,
30,Kaele,CM,87.87,overcast clouds,10.1092,14.4508,
38,Ujjain,IN,86.79,overcast clouds,23.1833,75.7667,
39,Victoria,HK,82.33,scattered clouds,22.2855,114.1577,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["Hotel Name"] ==""] = np.nan
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavaratti,IN,82.85,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
7,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Atuona,PF,79.70,broken clouds,-9.8000,-139.0333,Villa Enata
16,Mahibadhoo,MV,83.03,overcast clouds,3.7833,72.9667,Omadhoo Inn
22,Rikitea,PF,76.71,overcast clouds,-23.1203,-134.9692,Pension Maro'i
24,Novo Aripuana,BR,84.69,overcast clouds,-5.1206,-60.3797,Hotel Ezequiel
28,Praia,CV,77.54,few clouds,14.9215,-23.5087,LT Aparthotel
30,Kaele,CM,87.87,overcast clouds,10.1092,14.4508,Hôtel Les Palmiers
38,Ujjain,IN,86.79,overcast clouds,23.1833,75.7667,SPOT ON 69320 Hotel New Neha Palace
39,Victoria,HK,82.33,scattered clouds,22.2855,114.1577,Mini Hotel Central


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))